<a href="https://colab.research.google.com/github/NonMundaneDev/layer-ml-pipelines-spark/blob/main/Colab_Jupyter_NB_Customer_Income.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# If you are using Colab...

In [ ]:
!python --version

Python 3.7.11


In [ ]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark

!pip install pyspark==3.0.2

     |████████████████████████████████| 204.8 MB 56 kB/s 
     |████████████████████████████████| 198 kB 49.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.2-py2.py3-none-any.whl size=205186690 sha256=9c09aeae06e951f7a44bee90bb6c5a78a80f7146c2de61f9538f30c80172faeb
  Stored in directory: /root/.cache/pip/wheels/9a/39/f6/970565f38054a830e9a8593f388b36e14d75dba6c6fdafc1ec
Successfully built pyspark


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-09-05 03:50:20--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 18.205.222.128, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  3.05MB/s    in 5.4s    

2021-09-05 03:50:26 (2.43 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://f291-35-203-142-45.ngrok.io


# If you are using Local Jupyter Notebook

You can learn how to install PySpark on Jupyter Notebook in [this video](https://youtu.be/K_aLjzVySvk).

Learn how to install [MLeap](https://github.com/combust/mleap) on Jupyter Notebook in [this guide](https://combust.github.io/mleap-docs/notebooks/jupyter.html).

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = SparkContext(conf=conf)
#spark = SparkSession.builder.getOrCreate()

In [ ]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

# Framing The Problem



## Description

An organization seeks a new marketing opportunity with a newly released product. They realize they could target a segment of their users above a certain income level with marketing ads, which will increase the likelihood that the customer will purchase the product.

The team suggests that you build an application to integrate with the existing marketing tool to send relevant emails to customers that the marketing team has designed. At this stage, the application should be correct often than not.

Dataset on prior customers and their income levels is available to be explored and used to build the ML application. 

From the description, the problem does seem like a supervised learning and classification problem. It will be further confirmed when you explore the dataset.

# Dataset Exploration

<b>Dataset location: </b>https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data


##Features

`age`: Age of the adult.

`workclass`: Working class status of adult (Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked).

`fnlwgt` (or Final Weight): Census-specific column that represents the weight given to the record based on the estimate of the number of people that particular adult observation represents.

`education`: Level of education of adult (Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool).

`education-num` (or Education number of years): Number of educational years completed by the adult.

`marital-status`: Marital status of the adult (Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse).

`occupation`: Adult's work occupation (Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces).

`relationship`: Adult's relationship status (Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried).

`race`: Adult's race (White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black).

`sex`: Adult's gender (Female, Male).

`capital-gain`: Monetary capital gains for the adult.

`capital-loss`: Monetary capital losses for the adult.

`hours-per-week`: Average hours per week worked.

`native-country`: Native country (United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands).

##Target Variable
`income`: Income class (<=50K, >50K).

This is already looking like a binary classification problem and a common theme with binary classification problem is class imbalance, which you will look out for as you validate and explore the data.

# Developing The ML Pipeline

## Steps In Developing The ML Pipeline (Training Pipeline)

1. Data ingestion.

1. Data validation.

2. Data preparation.

3. Data segregation (splitting).

4. Model development

 - Model training.

 - Model selection.

 - Model evaluation.

 - Model validation.

### Import necessary libraries.

In [ ]:
import pyspark.sql.functions as func
from pyspark.sql.functions import *

from pyspark.sql.types import (
    FloatType, 
    IntegerType
)



from pyspark.ml.feature import (
    VectorAssembler,
    MinMaxScaler,
    OneHotEncoder,
    StringIndexer
)

from pyspark.ml.classification import (
    LogisticRegression,
    DecisionTreeClassifier,
    RandomForestClassifier,
    GBTClassifier
)

from pyspark.ml import Pipeline

from pyspark.ml.tuning import (
    ParamGridBuilder,
    CrossValidator,
)

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
# Download the dataset
!wget -q https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data

### Initialize the Spark Session to start a Spark application

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Classifying if a customer's income is >$50k or lesser.") \
    .getOrCreate()


## Data Ingestion

1. Read the dataset in a csv format.
2. Do not include an header.
3. Remove the leading whitespace for each value
4. Read the directory of the text file.

In [ ]:
rawData = spark.read.format('csv') .option('header', 'false') \
                .option('ignoreLeadingWhiteSpace', 'true').load('adult.data')

In [ ]:
# You'll specify the features and label you  want to use for the dataset
dataset = rawData.toDF('Age',
               'WorkClass',
               'FnlWgt',
               'Education',
               'EducationNum',
               'MaritalStatus',
               'Occupation',
               'Relationship',
               'Race',
               'Gender',
               'CapitalGain',
               'CapitalLoss',
               'HoursPerWeek',
               'NativeCountry',
               'Income'
                )

## Data Validation

Count the number of observations in the dataset...

In [ ]:
dataset.count()

32561

Validate that your dataset is what you expect. You should not do this if your dataset is large, as it will become inefficient.

In [ ]:
dataset.toPandas().head()

,Age,WorkClass,FnlWgt,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Print the schema of the dataset to understand the structure...

In [ ]:
dataset.printSchema()

root
 |-- Age: string (nullable = true)
 |-- WorkClass: string (nullable = true)
 |-- FnlWgt: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- EducationNum: string (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Relationship: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- CapitalGain: string (nullable = true)
 |-- CapitalLoss: string (nullable = true)
 |-- HoursPerWeek: string (nullable = true)
 |-- NativeCountry: string (nullable = true)
 |-- Income: string (nullable = true)



The `FnlWgt` (final weight) feature does not logically have any effect on the problem we are trying to solve so we can drop the column as it does not appear meaningful.

In [ ]:
dataset = dataset.drop('FnlWgt')

Examine the dataset and confirm the `FnlWgt` column has been dropped...

In [ ]:
dataset.toPandas()

,Age,WorkClass,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


Check for missing values...

In [ ]:
dataset.select(*(sum(col(c).isNull().cast("int")).alias(c) \
                 for c in dataset.columns)).show()

+---+---------+---------+------------+-------------+----------+------------+----+------+-----------+-----------+------------+-------------+------+
|Age|WorkClass|Education|EducationNum|MaritalStatus|Occupation|Relationship|Race|Gender|CapitalGain|CapitalLoss|HoursPerWeek|NativeCountry|Income|
+---+---------+---------+------------+-------------+----------+------------+----+------+-----------+-----------+------------+-------------+------+
|  0|        0|        0|           0|            0|         0|           0|   0|     0|          0|          0|           0|            0|     0|
+---+---------+---------+------------+-------------+----------+------------+----+------+-----------+-----------+------------+-------------+------+



Looks like there is none, but careful, missing values in the data can also be represented by '?'

Let's check for "?" in each column of the data.

In [ ]:
for col in dataset.columns:
  print(col, "\t", "with '?' values:", dataset.filter(dataset[col]=="?").count())

Age 	 with '?' values: 0
WorkClass 	 with '?' values: 1836
Education 	 with '?' values: 0
EducationNum 	 with '?' values: 0
MaritalStatus 	 with '?' values: 0
Occupation 	 with '?' values: 1843
Relationship 	 with '?' values: 0
Race 	 with '?' values: 0
Gender 	 with '?' values: 0
CapitalGain 	 with '?' values: 0
CapitalLoss 	 with '?' values: 0
HoursPerWeek 	 with '?' values: 0
NativeCountry 	 with '?' values: 583
Income 	 with '?' values: 0


You can directly inspect the particular columns with "?"...

In [ ]:
cols = ["WorkClass", "Occupation", "NativeCountry"]

for col in cols:
  print(dataset.select(col).distinct().rdd.map(lambda r: r[0]).collect())
  print("\n")

['Self-emp-not-inc', 'Local-gov', 'State-gov', 'Private', 'Without-pay', 'Federal-gov', 'Never-worked', '?', 'Self-emp-inc']


['Sales', 'Exec-managerial', 'Prof-specialty', 'Handlers-cleaners', 'Farming-fishing', 'Craft-repair', 'Transport-moving', 'Priv-house-serv', 'Protective-serv', 'Other-service', 'Tech-support', 'Machine-op-inspct', 'Armed-Forces', '?', 'Adm-clerical']


['Philippines', 'Germany', 'Cambodia', 'France', 'Greece', 'Taiwan', 'Ecuador', 'Nicaragua', 'Hong', 'Peru', 'India', 'China', 'Italy', 'Holand-Netherlands', 'Cuba', 'South', 'Iran', 'Ireland', 'Thailand', 'Laos', 'El-Salvador', 'Mexico', 'Guatemala', 'Honduras', 'Yugoslavia', 'Puerto-Rico', 'Jamaica', 'Canada', 'United-States', 'Dominican-Republic', 'Outlying-US(Guam-USVI-etc)', 'Japan', 'England', 'Haiti', 'Poland', 'Portugal', '?', 'Columbia', 'Scotland', 'Hungary', 'Vietnam', 'Trinadad&Tobago']




Convert missing values represented by "?" to null values so they can be treated as missing values...

In [ ]:
dataset = dataset.replace('?', None)

Run a prior code again for check missing values in your data now, as you have replaced "?" with null values...

In [ ]:
#from pyspark.sql.functions import col, sum
dataset.select(*(sum(col(c).isNull().cast("int")).alias(c) \
                 for c in dataset.columns)).show()

+---+---------+---------+------------+-------------+----------+------------+----+------+-----------+-----------+------------+-------------+------+
|Age|WorkClass|Education|EducationNum|MaritalStatus|Occupation|Relationship|Race|Gender|CapitalGain|CapitalLoss|HoursPerWeek|NativeCountry|Income|
+---+---------+---------+------------+-------------+----------+------------+----+------+-----------+-----------+------------+-------------+------+
|  0|     1836|        0|           0|            0|      1843|           0|   0|     0|          0|          0|           0|          583|     0|
+---+---------+---------+------------+-------------+----------+------------+----+------+-----------+-----------+------------+-------------+------+



You can see there are now missing values in the 3 columns we performed operations on. Drop all rows which contain even a single missing value
The value `'any'` for parameter how specifies that even a single missing value in a row should result in it being dropped (as opposed to `'all'` where all values need to be missing).

In [ ]:
dataset = dataset.dropna(how='any')

Count observations in dataset...

In [ ]:
dataset.count()

30162

That's about 2,000+ observations dropped. This should likely not affect your analysis.

You can inspect the DataFrame...


In [ ]:
dataset.toPandas()

,Age,WorkClass,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30157,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
30158,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
30159,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
30160,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


## (Optional) Data Exploration

Explore the data to check for potential class imbalance, as it's a classification problem--especially a binary classification problem.

In [ ]:
IncomeDF = dataset.groupBy('Income').count()\
              .select(func.col("Income")\
              .alias("distinct_Income"),\
              ((func.col("count")/dataset.count())*100)\
              .alias("Income_percentage"))

# Round up the percentage to the nearest 2 decimal places.
IncomeDF_per = IncomeDF.withColumn("Income_%", \
                                   round(IncomeDF["Income_percentage"], 2))

# Drop the original percentage column
IncomeDF_per = IncomeDF_per.drop("Income_percentage")

# Show the income level percentage
IncomeDF_per.show()          

+---------------+--------+
|distinct_Income|Income_%|
+---------------+--------+
|          <=50K|   75.11|
|           >50K|   24.89|
+---------------+--------+




The classes are imbalanced, with a skew toward the ‘<=50K‘ class label.

- ‘>50K’ (the hig income class for adults in this census) is right around 25%.
- ‘<=50K’ (the minority income class for adults in this census) is right around 75%.

You want to ensure model performs higher than 75% in terms of performance, as this is the baseline that indicates your model (if deployed) may actually provide any sort of value to the marketing department.

Of course, the closer to a 100% the model performs, the better. But this will inevitably be determined by what performance metric you choose to use. In this case, given that the class imbalance is not severe and that both class labels are equally important, it is common to use classification accuracy or classification error to report model performance on this dataset.

This means techniques for imbalanced classification can be used whilst model performance can still be reported using classification accuracy, as is used with balanced classification problems.

## Data Preparation/Feature Engineering

From the dataset schema, you could see that all the features and label have a `String` datatype. However, there are some features that are meant to be numeric (`Float`) datatypes (for better precision).

Some of these include: `Age`, `EducationNum`, `CapitalGain`, `CapitalLoss`, `HoursPerWeek`.

In [ ]:

dataset = dataset.withColumn('Age', 
                             dataset['Age'].cast(FloatType()))
dataset = dataset.withColumn('EducationNum', 
                             dataset['EducationNum'].cast(FloatType()))
dataset = dataset.withColumn('CapitalGain', 
                             dataset['CapitalGain'].cast(FloatType()))
dataset = dataset.withColumn('CapitalLoss', 
                             dataset['CapitalLoss'].cast(FloatType()))
dataset = dataset.withColumn('HoursPerWeek', 
                             dataset['HoursPerWeek'].cast(FloatType()))

dataset.toPandas().head()

,Age,WorkClass,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income
0,39.0,State-gov,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50.0,Self-emp-not-inc,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38.0,Private,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53.0,Private,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28.0,Private,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


Now we can check the datatypes to see if our features were converted...

In [ ]:
dataset.schema.fields

# You can also do this with:
#dataset.dtypes

[StructField(Age,FloatType,true),
 StructField(WorkClass,StringType,true),
 StructField(Education,StringType,true),
 StructField(EducationNum,FloatType,true),
 StructField(MaritalStatus,StringType,true),
 StructField(Occupation,StringType,true),
 StructField(Relationship,StringType,true),
 StructField(Race,StringType,true),
 StructField(Gender,StringType,true),
 StructField(CapitalGain,FloatType,true),
 StructField(CapitalLoss,FloatType,true),
 StructField(HoursPerWeek,FloatType,true),
 StructField(NativeCountry,StringType,true),
 StructField(Income,StringType,true)]

The schema now contain only `StringType` and `FloatType` fields.

Spark ML offers four  different kinds of libraries to perform feature engineering:

- [Feature extractors](https://spark.apache.org/docs/2.2.3/ml-features.html#feature-extractors): Extracting features from “raw” data.

- [Feature transfomers](https://spark.apache.org/docs/2.2.3/ml-features.html#feature-transformers): Scaling, converting, or modifying features.

- [Feature selectors](https://spark.apache.org/docs/2.2.3/ml-features.html#feature-selectors): Selecting a subset from a larger set of features.

- [Locality Sensitive Hashing (LSH)](https://spark.apache.org/docs/2.2.3/ml-features.html#locality-sensitive-hashing): This class of algorithms combines aspects of feature transformation with other algorithms.

[Source](https://spark.apache.org/docs/2.2.3/ml-features.html).


### Feature Transformation

Start off by transforming the categorical fields. You will first use the [`StringIndexer`](https://spark.apache.org/docs/2.0.0-preview/ml-features.html#stringindexer) feature transformer to convert categorical values to indices, and then [one-hot-encode](https://en.wikipedia.org/wiki/One-hot#Machine_learning_and_statistics) them to integers.

In [ ]:
indexedDF = StringIndexer(
    inputCol='WorkClass', 
    outputCol='WorkClass_index').fit(dataset).transform(dataset)

Inspect the effect of the transformer...

In [ ]:
indexedDF.toPandas().head()

,Age,WorkClass,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income,WorkClass_index
0,39.0,State-gov,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K,3.0
1,50.0,Self-emp-not-inc,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K,1.0
2,38.0,Private,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K,0.0
3,53.0,Private,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K,0.0
4,28.0,Private,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K,0.0


You can see that a new column called `WorkClass_index` (which stores the indexed values of WorkClass) is created and appended to the end of the DataFrame. This confirms that **transformers do not change the DataFrame but append the transformations instead**.


You will now use the new indexed field to obtain a one-hot-encoded field:

In [ ]:
OHE = OneHotEncoder(
    inputCol="WorkClass_index", 
    outputCol="WorkClass_encoded")

encodedDF = OHE.fit(indexedDF)
encodedDF = encodedDF.transform(indexedDF)

The `WorkClass_encoded` field is created.
* This contains the one-hot-encoding for the `WorkClass` variable.
* This cannot operate directly on a column with string values - values need to be numeric. Hence we use the `WorkClass_index` as input.

In [ ]:
encodedDF.toPandas().head()

,Age,WorkClass,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income,WorkClass_index,WorkClass_encoded
0,39.0,State-gov,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K,3.0,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0)"
1,50.0,Self-emp-not-inc,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
2,38.0,Private,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)"
3,53.0,Private,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)"
4,28.0,Private,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)"


If you scroll to the right, you will observe once again that the transformations were appended to the column.

To view the original and transformed features together, perform the operation:

In [ ]:
encodedDF.select('WorkClass', 'WorkClass_index', 'WorkClass_encoded')\
         .toPandas()\
         .head()

,WorkClass,WorkClass_index,WorkClass_encoded
0,State-gov,3.0,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0)"
1,Self-emp-not-inc,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
2,Private,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)"
3,Private,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)"
4,Private,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)"


### Transform the entire dataset
* So far we have only transformed a single column
* We need to perform this transformation for every categorical and non-numeric column
* This will be simplified by using a Pipeline (a feature of Spark ML)

####  First, split the data into training and test sets

In [ ]:
(trainingData, testData) = dataset.randomSplit([0.8,0.2])

#### Encode all the categorical fields in the dataset
We begin by listing all the categorical fields

In [ ]:
categoricalFeatures = [item[0] for item in dataset.dtypes if \
                     item[1].startswith('string')]

#Select all columns except "Income" (label) which is also a string
categoricalFeatures = categoricalFeatures[:-1] 

print(categoricalFeatures)

['WorkClass', 'Education', 'MaritalStatus', 'Occupation', 'Relationship', 'Race', 'Gender', 'NativeCountry']


#### Create an array of StringIndexers to convert the categorical values to indices

In [ ]:
indexers = [StringIndexer(
    inputCol=column, 
    outputCol=column + '_index', 
    handleInvalid='keep') for column in categoricalFeatures]

#### Create an array of OneHotEncoders to encode the categorical values

In [ ]:
encoders = [OneHotEncoder(
    inputCol=column + '_index', 
    outputCol= column + '_encoded') for column in categoricalFeatures]

#### Scale the numeric fields

In [ ]:
# Select the numeric features
numFeatures = [item[0] for item in dataset.dtypes \
               if item[1].startswith('float')]

print(numFeatures)

['Age', 'EducationNum', 'CapitalGain', 'CapitalLoss', 'HoursPerWeek']


To scale numeric features, you will need to convert them to a single feature vector before applying a scaler.

In [ ]:
vectorFeatures = [VectorAssembler(inputCols=[col], \
                                  outputCol=col + "_vec") for col in numFeatures]

numScaler = [MinMaxScaler(
    inputCol= column + "_vec", 
    outputCol= column + '_scaled') for column in numFeatures]

#### Index the Label field

As you saw earlier, the target variable has two classes: `>= can also see that the target variable is represented using strings. This column will need to be label encoded with 0 for the majority class and 1 for the minority class, as is the custom for binary imbalanced classification tasks.

In [ ]:
labelIndexer = [StringIndexer(
    inputCol='Income', outputCol='Income_index')]

#### Create a pipeline
The pipeline contains the array of StringIndexers and OneHotEncoders

In [ ]:

pipeline = Pipeline(stages=indexers + encoders + vectorFeatures \
                    + numScaler + labelIndexer)

#### View the result of the transformations performed by this pipeline
This pipeline can transform our dataset into a format which can be used by our model

In [ ]:
transformedDF = pipeline.fit(trainingData).transform(trainingData)
transformedDF.toPandas().tail()

,Age,WorkClass,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income,WorkClass_index,Education_index,MaritalStatus_index,Occupation_index,Relationship_index,Race_index,Gender_index,NativeCountry_index,WorkClass_encoded,Education_encoded,MaritalStatus_encoded,Occupation_encoded,Relationship_encoded,Race_encoded,Gender_encoded,NativeCountry_encoded,Age_vec,EducationNum_vec,CapitalGain_vec,CapitalLoss_vec,HoursPerWeek_vec,Age_scaled,EducationNum_scaled,CapitalGain_scaled,CapitalLoss_scaled,HoursPerWeek_scaled,Income_index
24183,90.0,Private,Some-college,10.0,Never-married,Other-service,Not-in-family,Asian-Pac-Islander,Male,0.0,0.0,35.0,United-States,<=50K,0.0,1.0,1.0,5.0,1.0,2.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[90.0],[10.0],[0.0],[0.0],[35.0],[1.0],[0.6],[0.0],[0.0],[0.3469387755102041],0.0
24184,90.0,Private,Some-college,10.0,Separated,Adm-clerical,Own-child,White,Female,0.0,0.0,40.0,Puerto-Rico,<=50K,0.0,1.0,3.0,3.0,2.0,0.0,1.0,4.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0)","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",[90.0],[10.0],[0.0],[0.0],[40.0],[1.0],[0.6],[0.0],[0.0],[0.39795918367346933],0.0
24185,90.0,Self-emp-not-inc,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,10566.0,0.0,50.0,United-States,<=50K,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[90.0],[13.0],[10566.0],[0.0],[50.0],[1.0],[0.8],[0.1056610566105661],[0.0],[0.49999999999999994],0.0
24186,90.0,Self-emp-not-inc,HS-grad,9.0,Never-married,Exec-managerial,Not-in-family,White,Male,2964.0,0.0,12.0,United-States,<=50K,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[90.0],[9.0],[2964.0],[0.0],[12.0],[1.0],[0.5333333333333333],[0.02964029640296403],[0.0],[0.11224489795918366],0.0
24187,90.0,Self-emp-not-inc,Some-college,10.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,40.0,United-States,<=50K,1.0,1.0,0.0,9.0,0.0,0.0,0.0,0.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[90.0],[10.0],[0.0],[0.0],[40.0],[1.0],[0.6],[0.0],[0.0],[0.39795918367346933],0.0


#### Select the required features
At this point the dataset contains a lot of additional columns. We select the features needed by our model

In [ ]:
requiredFeatures = [
    'Age_scaled',
    'EducationNum_scaled',
    'CapitalGain_scaled',
    'CapitalLoss_scaled',
    'HoursPerWeek_scaled',
    'WorkClass_encoded',
    'Education_encoded',
    'MaritalStatus_encoded',
    'Occupation_encoded',
    'Relationship_encoded',
    'Race_encoded',
    'Gender_encoded',
    'NativeCountry_encoded'
]

#### VectorAssembler
VectorAssembler is a transformer that combines a given list of columns into a single vector column. It is useful for combining raw features and features generated by different feature transformers into a single feature vector
* We had previously written our own function to create such a vector

In [ ]:
assembler = VectorAssembler(inputCols=requiredFeatures, outputCol='features')

In [ ]:
transformedDF = assembler.transform(transformedDF)
transformedDF.toPandas().tail()

,Age,WorkClass,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income,WorkClass_index,Education_index,MaritalStatus_index,Occupation_index,Relationship_index,Race_index,Gender_index,NativeCountry_index,WorkClass_encoded,Education_encoded,MaritalStatus_encoded,Occupation_encoded,Relationship_encoded,Race_encoded,Gender_encoded,NativeCountry_encoded,Age_vec,EducationNum_vec,CapitalGain_vec,CapitalLoss_vec,HoursPerWeek_vec,Age_scaled,EducationNum_scaled,CapitalGain_scaled,CapitalLoss_scaled,HoursPerWeek_scaled,Income_index,features
24183,90.0,Private,Some-college,10.0,Never-married,Other-service,Not-in-family,Asian-Pac-Islander,Male,0.0,0.0,35.0,United-States,<=50K,0.0,1.0,1.0,5.0,1.0,2.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[90.0],[10.0],[0.0],[0.0],[35.0],[1.0],[0.6],[0.0],[0.0],[0.3469387755102041],0.0,"(1.0, 0.6, 0.0, 0.0, 0.3469387755102041, 1.0, ..."
24184,90.0,Private,Some-college,10.0,Separated,Adm-clerical,Own-child,White,Female,0.0,0.0,40.0,Puerto-Rico,<=50K,0.0,1.0,3.0,3.0,2.0,0.0,1.0,4.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0)","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",[90.0],[10.0],[0.0],[0.0],[40.0],[1.0],[0.6],[0.0],[0.0],[0.39795918367346933],0.0,"(1.0, 0.6, 0.0, 0.0, 0.39795918367346933, 1.0,..."
24185,90.0,Self-emp-not-inc,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,10566.0,0.0,50.0,United-States,<=50K,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[90.0],[13.0],[10566.0],[0.0],[50.0],[1.0],[0.8],[0.1056610566105661],[0.0],[0.49999999999999994],0.0,"(1.0, 0.8, 0.1056610566105661, 0.0, 0.49999999..."
24186,90.0,Self-emp-not-inc,HS-grad,9.0,Never-married,Exec-managerial,Not-in-family,White,Male,2964.0,0.0,12.0,United-States,<=50K,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[90.0],[9.0],[2964.0],[0.0],[12.0],[1.0],[0.5333333333333333],[0.02964029640296403],[0.0],[0.11224489795918366],0.0,"(1.0, 0.5333333333333333, 0.02964029640296403,..."
24187,90.0,Self-emp-not-inc,Some-college,10.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,40.0,United-States,<=50K,1.0,1.0,0.0,9.0,0.0,0.0,0.0,0.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[90.0],[10.0],[0.0],[0.0],[40.0],[1.0],[0.6],[0.0],[0.0],[0.39795918367346933],0.0,"(1.0, 0.6, 0.0, 0.0, 0.39795918367346933, 0.0,..."


In [ ]:
transformedDF.select('features').toPandas().tail()

,features
24183,"(1.0, 0.6, 0.0, 0.0, 0.3469387755102041, 1.0, ..."
24184,"(1.0, 0.6, 0.0, 0.0, 0.39795918367346933, 1.0,..."
24185,"(1.0, 0.8, 0.1056610566105661, 0.0, 0.49999999..."
24186,"(1.0, 0.5333333333333333, 0.02964029640296403,..."
24187,"(1.0, 0.6, 0.0, 0.0, 0.39795918367346933, 0.0,..."


## Model Selection

- [Binomial logistic regression](https://spark.apache.org/docs/latest/ml-classification-regression.html#binomial-logistic-regression) --> For predicting a binary class problem.

 - Assumption for selecting this estimator: To establish a baseline performance for the classifiers on the dataset.
 - Learn more about LR in the [docs](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.LogisticRegression.html).

- [Decision tree classifier](https://spark.apache.org/docs/latest/ml-classification-regression.html#decision-tree-classifier): Decision trees are great for giving some boost in performance but also risk overfitting and are mostly unstable for production usage.

 - Learn more about this estimator in the [docs](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.DecisionTreeClassifier.html#pyspark.ml.classification.DecisionTreeClassifier).


- [Random forest classifier](https://spark.apache.org/docs/latest/ml-classification-regression.html#random-forest-classifier): To curb any potential for overfitting on the dataset, and it's a really good candidate for an optimal production model.
 - If we deploy this to production, it can be able to handle missing values from new data it needs to predict on.
 - Learn more about this estimator in the [docs](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.ml.classification.RandomForestClassifier.html).


- (Optional) [Gradient-boosted tree classifier](https://spark.apache.org/docs/latest/ml-classification-regression.html#gradient-boosted-tree-classifier): An alternative to random forest, if it performs better.
 - Learn more about this estimator in the [docs](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.GBTClassifier.html).



In [ ]:
# lr = LogisticRegression(labelCol='Income_index', featuresCol='features')

# dt = DecisionTreeClassifier(labelCol="Income_index", \
#                             featuresCol="features")

# rf = RandomForestClassifier(labelCol='Income_index', 
#                             featuresCol='features')

gbt = GBTClassifier(labelCol="Income_index", featuresCol="features")


In [ ]:
# pipeline_lr = Pipeline(
#     stages=indexers + encoders + vectorFeatures + numScaler + labelIndexer \
#      + [assembler, lr]
# )

# pipeline_dt = Pipeline(
#     stages=indexers + encoders + vectorFeatures + numScaler + labelIndexer \
#      + [assembler, dt]
# )

# pipeline_rf = Pipeline(
#     stages=indexers + encoders + vectorFeatures + numScaler + labelIndexer \
#      + [assembler, rf]
# )

pipeline_gbt = Pipeline(
    stages=indexers + encoders + vectorFeatures + numScaler + labelIndexer \
     + [assembler, gbt]
)

I could have easily defined the entire parameter grid for all esimators in one builder, but that would be quite compute intensive and may take a lot of time to run (on an instance like that of G Colab), so I will try building and using them one-by-one...

In [ ]:

# paramGrid_lr = ParamGridBuilder().addGrid(
#     lr.maxIter, [10,50,100]).addGrid(
#     lr.regParam, [0.1, 0.3, 1.0]).addGrid(
#     lr.elasticNetParam, [0.0, 0.7, 1.0]).addGrid(
#     lr.threshold, [0.5, 0.75]
#     ).build()


# paramGrid_dt = ParamGridBuilder().addGrid(
#     dt.maxDepth, [5, 10, 15, 20]).build()


# paramGrid_rf = ParamGridBuilder().addGrid(
#     rf.maxDepth, [5, 10, 15]).addGrid(
#     rf.numTrees, [50, 100, 150]).build()


paramGrid_gbt = ParamGridBuilder().addGrid(
    gbt.maxDepth, [5, 10, 15]).addGrid(
    gbt.maxIter, [20, 25, 50]).build()

In [ ]:
evaluator = MulticlassClassificationEvaluator(
    labelCol='Income_index', 
    predictionCol='prediction', 
    metricName='accuracy')

LEARN HOW TO SAVE AND LOAD SPARK ML PIPELINES LOCALLY [HERE](https://stackoverflow.com/questions/44981407/pyspark-ml-how-to-save-pipeline-and-randomforestclassificationmodel).

#### Define the CrossValidator
This is used to put all the pieces together
* <b>estimator: </b>Can be a standalone estimator or a pipeline with an estimator at the end. We use our pipeline
* <b>estimatorParamMaps: </b>We add our paramGrid in order to build models with different combinations of the parameters
* <b>evaluator: </b>To evaluate each model, we specify our evaluator

In [ ]:

# crossval_lr = CrossValidator(estimator=pipeline_lr,
#                           estimatorParamMaps=paramGrid_lr,
#                           evaluator=evaluator,
#                           numFolds=3)

# crossval_dt = CrossValidator(estimator=pipeline_dt,
#                           estimatorParamMaps=paramGrid_dt,
#                           evaluator=evaluator,
#                           numFolds=3)

# crossval_rf = CrossValidator(estimator=pipeline_rf,
#                           estimatorParamMaps=paramGrid_rf,
#                           evaluator=evaluator,
#                           numFolds=3)

crossval_gbt = CrossValidator(estimator=pipeline_gbt,
                          estimatorParamMaps=paramGrid_gbt,
                          evaluator=evaluator,
                          numFolds=3)

In [ ]:
# model_lr = crossval_lr.fit(trainingData)

# model_dt = crossval_dt.fit(trainingData)

# model_rf = crossval_rf.fit(trainingData)

model_gbt = crossval_gbt.fit(trainingData)

In [ ]:
# lrModel = model_lr.bestModel.stages[-1]
# lrModel

# dtModel = model_dt.bestModel.stages[-1]
# dtModel

# rfModel = model_rf.bestModel.stages[-1]
# rfModel

gbtModel = model_gbt.bestModel.stages[-1]
gbtModel

GBTClassificationModel: uid = GBTClassifier_85891fc37465, numTrees=50, numClasses=2, numFeatures=103

In [ ]:
predictions = model_gbt.transform(testData)
predictionsDF = predictions.toPandas()
predictionsDF.head()

,Age,WorkClass,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income,WorkClass_index,Education_index,MaritalStatus_index,Occupation_index,Relationship_index,Race_index,Gender_index,NativeCountry_index,WorkClass_encoded,Education_encoded,MaritalStatus_encoded,Occupation_encoded,Relationship_encoded,Race_encoded,Gender_encoded,NativeCountry_encoded,Age_vec,EducationNum_vec,CapitalGain_vec,CapitalLoss_vec,HoursPerWeek_vec,Age_scaled,EducationNum_scaled,CapitalGain_scaled,CapitalLoss_scaled,HoursPerWeek_scaled,Income_index,features,rawPrediction,probability,prediction
0,17.0,Local-gov,11th,7.0,Never-married,Prof-specialty,Own-child,Black,Male,0.0,0.0,15.0,United-States,<=50K,2.0,5.0,1.0,1.0,2.0,1.0,0.0,0.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[17.0],[7.0],[0.0],[0.0],[15.0],[0.0],[0.4],[0.0],[0.0],[0.14285714285714285],0.0,"(0.0, 0.4, 0.0, 0.0, 0.14285714285714285, 0.0,...","[1.8493557113817651, -1.8493557113817651]","[0.9758426205381941, 0.02415737946180585]",0.0
1,17.0,Local-gov,12th,8.0,Never-married,Adm-clerical,Own-child,White,Female,0.0,0.0,12.0,United-States,<=50K,2.0,11.0,1.0,3.0,2.0,0.0,1.0,0.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[17.0],[8.0],[0.0],[0.0],[12.0],[0.0],[0.4666666666666667],[0.0],[0.0],[0.11224489795918366],0.0,"(0.0, 0.4666666666666667, 0.0, 0.0, 0.11224489...","[1.8909834619733443, -1.8909834619733443]","[0.9777294305344049, 0.022270569465595136]",0.0
2,17.0,Private,10th,6.0,Never-married,Handlers-cleaners,Own-child,White,Male,0.0,0.0,20.0,United-States,<=50K,0.0,7.0,1.0,8.0,2.0,0.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[17.0],[6.0],[0.0],[0.0],[20.0],[0.0],[0.3333333333333333],[0.0],[0.0],[0.19387755102040816],0.0,"(0.0, 0.3333333333333333, 0.0, 0.0, 0.19387755...","[1.8890512549304643, -1.8890512549304643]","[0.9776451289695091, 0.02235487103049094]",0.0
3,17.0,Private,10th,6.0,Never-married,Handlers-cleaners,Own-child,White,Male,0.0,0.0,20.0,United-States,<=50K,0.0,7.0,1.0,8.0,2.0,0.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[17.0],[6.0],[0.0],[0.0],[20.0],[0.0],[0.3333333333333333],[0.0],[0.0],[0.19387755102040816],0.0,"(0.0, 0.3333333333333333, 0.0, 0.0, 0.19387755...","[1.8890512549304643, -1.8890512549304643]","[0.9776451289695091, 0.02235487103049094]",0.0
4,17.0,Private,10th,6.0,Never-married,Handlers-cleaners,Own-child,White,Male,0.0,0.0,24.0,United-States,<=50K,0.0,7.0,1.0,8.0,2.0,0.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[17.0],[6.0],[0.0],[0.0],[24.0],[0.0],[0.3333333333333333],[0.0],[0.0],[0.2346938775510204],0.0,"(0.0, 0.3333333333333333, 0.0, 0.0, 0.23469387...","[1.88905125493

In [ ]:
evaluator = MulticlassClassificationEvaluator(
    labelCol='Income_index', 
    predictionCol='prediction', 
    metricName='accuracy')

accuracy = evaluator.evaluate(predictions)
print(f"Model accuracy: {accuracy*100:.3f}%")

Model accuracy: 85.420%


Check for incorrect predictions...

In [ ]:
predictionsDF.loc[
    predictionsDF['Income_index'] != predictionsDF['prediction']
]

## Use MLeap to serialize pipeline

Colab does not work well with MLeap (`v =<1.18.0`) as MLeap will require Docker to be installed on local machine--which Colab does not support currently or plan on supporting anytime soon.


In [ ]:
# Replace gbtModel with appropriate model name

# SimpleSparkSerializer().serializeToBundle(
#     gbtModel, "jar:file:/tmp/model.zip", predictions
# )

#### Final Pipeline
* The pipeline we built previously only transformed the feature columns
* We re-create the pipeline to include the VectorAssembler and the estimator

The pipeline to be used to build the model contains all the transformers and ends with the estimator

# Credits

**Dataset Credits:**

The dataset is credited to Ronny Kohavi and Barry Becker (from [this paper](https://dl.acm.org/doi/10.5555/3001460.3001502)) and was drawn from the 1994 [United States Census Bureau](https://www.census.gov/) data and involves using personal details such as education level to predict whether an individual will earn more or less than $50,000 per year.

**Resources**

Concepts from this notebook are heavily inspired by Janani Ravi's course on ["Building Machine Learning Models in Spark 2"](https://app.pluralsight.com/library/courses/spark-2-building-machine-learning-models/table-of-contents).


**Other References**

- [Imbalanced Classification with the Adult Income Dataset](https://machinelearningmastery.com/imbalanced-classification-with-the-adult-income-dataset/) 

- [Apache Spark ML documentation](https://spark.apache.org/docs/2.0.0-preview/ml-guide.html).